## **Step 1.** Import packages

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

print('TF version:', tf.__version__)
print('TFP version:', tfp.__version__)

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from pathlib import Path

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **Step 2.** Load data

In [ ]:
datapath = Path('/kaggle/input/tabular-playground-series-apr-2022')

train_df = pd.read_csv(
    datapath / 'train.csv',
    usecols=['sequence', 'subject', 'step', 
             'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 
             'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 
             'sensor_10', 'sensor_11', 'sensor_12'],
    dtype={
        'sequence': 'uint16',
        'subject': 'uint16',
        'step': 'uint8',
        'sensor_00': 'float32',
        'sensor_01': 'float32',
        'sensor_02': 'float32',
        'sensor_03': 'float32',
        'sensor_04': 'float32',
        'sensor_05': 'float32',
        'sensor_06': 'float32',
        'sensor_07': 'float32',
        'sensor_08': 'float32',
        'sensor_09': 'float32',
        'sensor_10': 'float32',
        'sensor_11': 'float32',
        'sensor_12': 'float32',
    },
)

train_labels_df = pd.read_csv(
    datapath / 'train_labels.csv',
    usecols=['sequence', 'state'],
    dtype={
        'sequence': 'uint16',
        'state': 'uint8',
    },
)

test_df = pd.read_csv(
    datapath / 'test.csv',
    usecols=['sequence', 'subject', 'step', 
             'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 
             'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 
             'sensor_10', 'sensor_11', 'sensor_12'],
    dtype={
        'sequence': 'uint16',
        'subject': 'uint16',
        'step': 'uint8',
        'sensor_00': 'float32',
        'sensor_01': 'float32',
        'sensor_02': 'float32',
        'sensor_03': 'float32',
        'sensor_04': 'float32',
        'sensor_05': 'float32',
        'sensor_06': 'float32',
        'sensor_07': 'float32',
        'sensor_08': 'float32',
        'sensor_09': 'float32',
        'sensor_10': 'float32',
        'sensor_11': 'float32',
        'sensor_12': 'float32',
    },
)

print(train_df.shape)
print(test_df.shape)
print(train_labels_df.shape)

## **Step 3.** Prepare data

### **Step 3.1.** Set globals

In [ ]:
n_timesteps = 60 # sequence length
n_features = 13 # number of sensors
n_outputs = 2 # binary states 0 and 1

print('time steps per sequence : {}'.format(n_timesteps))
print('number of features : {}'.format(n_features))
print('number of output classes : {}'.format(n_outputs))

### **Step 3.2.** Check data

In [ ]:
print('total sequences : {}'.format(train_df['sequence'].nunique()))
print('total sequences missing steps : {}'.format((train_df.groupby('sequence').size() != 60).sum()))

print('total sequences : {}'.format(test_df['sequence'].nunique()))
print('total sequences missing steps : {}'.format((test_df.groupby('sequence').size() != 60).sum()))

### **Step 3.3.** Drop columns and Standardize

In [ ]:
filtered_train_df = train_df.drop(['sequence','subject','step'], axis=1)
filtered_test_df  =  test_df.drop(['sequence','subject','step'], axis=1)
filtered_train_labels_df  = train_labels_df.drop('sequence', axis=1)

print(filtered_train_df.shape)
print(filtered_test_df.shape)
print(filtered_train_labels_df.shape)

In [ ]:
# perform a robust scaler transform of the dataset
trans = StandardScaler()
train_data = trans.fit_transform(filtered_train_df)
test_data  = trans.transform(filtered_test_df)
# convert the array back to a dataframe
scaled_train_df = pd.DataFrame(train_data)
scaled_test_df = pd.DataFrame(test_data)

### **Step 3.4.** Reshape data

In [ ]:
X_train = scaled_train_df.values.reshape(25968, 60, 13)
X_test  =  scaled_test_df.values.reshape(12218, 60, 13)

print(X_train.shape)
print(X_test.shape)

In [ ]:
enc = OneHotEncoder()
y_train_oh = enc.fit_transform(filtered_train_labels_df).toarray()
y_train_oh.shape

In [ ]:
# Inspect some of the data by making plots

# Dictionary containing the labels and the associated states
label_to_state = {0: 'idle', 1: 'active'}

def make_plots(num_examples_per_category):
    for label in range(2):
        x_label = X_train[y_train_oh[:, 0] == label]
        for i in range(num_examples_per_category):
            fig, ax = plt.subplots(figsize=(10, 1))
            ax.imshow(x_label[100*i].T, cmap='Greys', vmin=-1, vmax=1)
            ax.axis('off')
            if i == 0:
                ax.set_title(label_to_state[label])
            plt.show()
        
make_plots(1)

## **Step 4.** Train model

### **Step 4.1.** Construct model

In [ ]:
model = Sequential([
                    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)),
                    Conv1D(filters=64, kernel_size=3, activation='relu'),
                    Dropout(0.5),
                    MaxPooling1D(pool_size=2),
                    Flatten(),
                    Dense(100, activation='relu'),
                    Dense(n_outputs, activation='softmax')
])

model.summary()

### **Step 4.2.** Compile and fit model

In [ ]:
epochs = 50
batch_size = 128

callbacks = [
    ModelCheckpoint(
        "/kaggle/working/best_model.h5", 
        save_best_only=True, monitor="val_loss"
    ),
    ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=15, min_lr=0.0001
    ),
    EarlyStopping(monitor="val_loss", patience=15, verbose=1),
]
model.compile(
    optimizer="adam",
    loss=BinaryCrossentropy(),
    metrics=["accuracy"],
)
history = model.fit(
    X_train,
    y_train_oh,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

## **Step 5.** Predict and push submission

In [ ]:
model.load_weights('/kaggle/working/best_model.h5')

y_pred = np.array(np.argmax(model.predict(X_test), axis=1))[...,np.newaxis]
y_pred.shape

In [ ]:
sequences = test_df['sequence'].unique().astype(int)[...,np.newaxis]
sequences.shape

In [ ]:
results = np.concatenate((sequences, y_pred.astype(int)), axis=1)
results_df = pd.DataFrame({'sequence': results[:,0], 'state': results[:,1]})
results_df

In [ ]:
results_df.to_csv('/kaggle/working/submission.csv', sep=',', encoding='utf-8', header=True, index=False)